In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib.pyplot import rc_context
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)
#sc.logging.print_versions() 
sc.logging.print_header()
sc.settings.verbosity = 3  
sc.settings.set_figure_params(dpi=120, color_map='viridis')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.4 scipy==1.9.3 pandas==1.5.1 scikit-learn==1.1.3 statsmodels==0.13.5 python-igraph==0.10.2 pynndescent==0.5.8


In [2]:
adata_gex=sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex.h5ad')

In [3]:
tcell= pd.read_csv('/nfs/team205/ld21/renal/T_NK_compartment_annotation_updated.csv', index_col=0)

In [4]:
adata = adata_gex[(adata_gex.obs.index.isin(tcell.index))]

In [5]:
adata.obs['final_anno']='uk'
adata.obs['final_anno']=adata.obs['final_anno'].astype('object')

/tmp/ipykernel_5599/501034239.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['final_anno']='uk'


In [6]:
adata.obs['final_anno'].update(pd.Series(tcell['level_2'].astype('object')))

In [7]:
adata.obs['final_anno'].unique()

array(['T reg', 'T CD4 Naive', 'T CD4 Helper', 'NK', 'doublets',
       'T CD8 CTL', 'T CD4 CTL', 'NK CD56 bright', 'T CD8 Mem', 'NKT',
       'T CD4 cycling', 'T CD8 Naive', 'MAIT', 'T g/d', 'T CD8 EMRA',
       'ILC'], dtype=object)

In [8]:
adata=adata[~adata.obs['final_anno'].isin(['doublets'])]

In [9]:
list(adata.obs['final_anno'].unique())

['T reg',
 'T CD4 Naive',
 'T CD4 Helper',
 'NK',
 'T CD8 CTL',
 'T CD4 CTL',
 'NK CD56 bright',
 'T CD8 Mem',
 'NKT',
 'T CD4 cycling',
 'T CD8 Naive',
 'MAIT',
 'T g/d',
 'T CD8 EMRA',
 'ILC']

In [10]:
'C139' in list(adata.obs['patient_id'].unique())

True

In [11]:
'C116' in list(adata.obs['patient_id'].unique())

True

In [12]:
'C141' in list(adata.obs['patient_id'].unique())

True

In [13]:
#remove C141 from the samples
adata = adata[~adata.obs['patient_id'].isin(['C141','C139','C116'])]

In [14]:
'C141' in list(adata.obs['patient_id'].unique())

False

In [15]:
'C139' in list(adata.obs['patient_id'].unique())

False

In [16]:
'C116' in list(adata.obs['patient_id'].unique())

False

In [18]:
# get all - not only monocyte
monocyte = adata[:]

In [19]:
list(monocyte.obs['final_anno'].unique())

['T reg',
 'T CD4 Naive',
 'NK',
 'T CD4 Helper',
 'T CD8 CTL',
 'T CD4 CTL',
 'NK CD56 bright',
 'T CD4 cycling',
 'T CD8 Naive',
 'MAIT',
 'T g/d',
 'NKT',
 'T CD8 Mem',
 'T CD8 EMRA',
 'ILC']

In [20]:
list(monocyte.obs['case_control'].unique())

['NEGATIVE', 'POSITIVE', 'NA', 'RECOVERY']

In [21]:
#remove 'nan' from case_control
monocyte = monocyte[monocyte.obs['case_control']!='NA']

In [22]:
list(monocyte.obs['case_control'].unique())

['NEGATIVE', 'POSITIVE', 'RECOVERY']

In [23]:
#getting p,n,r in wave 1 and wave 2

monocyte.obs['wave_case_control']='nan'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave1_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave1_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave1_recovery'

monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave2_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave2_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave2_recovery'

/tmp/ipykernel_5599/279369363.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  monocyte.obs['wave_case_control']='nan'


In [24]:
list(monocyte.obs['wave_case_control'].unique()) #wave 1 doesn't have recovery samples

['wave1_negative',
 'wave1_positive',
 'wave2_positive',
 'wave2_negative',
 'wave2_recovery']

In [ ]:
test = monocyte[monocyte.obs['wave_case_control']=='wave2_recovery']
print (list(test.obs['case_control'].unique()))
print (list(test.obs['centre'].unique()))
print (list(test.obs['sample_date'].unique()))

In [ ]:
df = pd.crosstab(monocyte.obs['wave_case_control'],monocyte.obs['final_anno'])
df

In [ ]:
#df.columns = df.columns.add_categories(['wave_case_control'])
df['wave_case_control'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='wave_case_control', kind='bar', stacked=True,
        title='distribution of monocytes in Wave 1 and Wave 2')
p1.set_ylabel("num of cells")
p1.set_xlabel(" ")
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
#getting percentage
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte.obs['wave_case_control'].value_counts()

In [ ]:
p2 = df.plot(x='wave_case_control', kind='bar', stacked=True,
        title='distribution of monocytes in Wave1 and Wave2',figsize=(5,5))
p2.set_ylabel('percentage')
p2.set_xlabel('')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  




p2.text(-0.2,101,'23749',fontsize=9)
p2.text(0.8,101,'22846',fontsize=9)
p2.text(1.8,101,'5364',fontsize=9)
p2.text(2.8,101,'34215',fontsize=9)
p2.text(3.8,101,'6008',fontsize=9)

# Getting wave 2 only 

In [25]:
monocyte_wave1 = monocyte[monocyte.obs['centre']=='NCL']
monocyte_wave2 = monocyte[monocyte.obs['centre']=='Cambridge']

In [26]:
monocyte_wave2.obs['case_control'].unique()

['POSITIVE', 'NEGATIVE', 'RECOVERY']
Categories (3, object): ['NEGATIVE', 'POSITIVE', 'RECOVERY']

In [27]:
# adding steroid status and case_control conditions

monocyte_wave2.obs['steroid_status']='nan'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'steroid_negative'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'steroid_recovery'

#####################

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'no_steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'no_steroid_negative'


monocyte_wave2.obs.loc[(monocyte_wave2.obs["patient_id"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'no_steroid_recovery'



/tmp/ipykernel_5599/948781802.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  monocyte_wave2.obs['steroid_status']='nan'


In [28]:
list(monocyte_wave2.obs['steroid_status'].unique())

['no_steroid_positive',
 'no_steroid_negative',
 'steroid_positive',
 'no_steroid_recovery',
 'steroid_recovery',
 'steroid_negative']

In [ ]:
print (list(monocyte_wave2.obs['sample_date'].unique()))

In [ ]:
test = monocyte_wave2[monocyte_wave2.obs['steroid_status']=='no_steroid_negative']
print (test.obs['case_control'].unique())
print (test.obs['sample_date'].unique())

In [ ]:
test = monocyte_wave2[monocyte_wave2.obs['steroid_status']=='steroid_recovery']
print (test.obs['case_control'].unique())
print (test.obs['sample_date'].unique())
print(test.obs['sample_id_broad'].unique())

In [ ]:
df = pd.crosstab(monocyte_wave2.obs['steroid_status'],monocyte_wave2.obs['annot4'])
df

In [ ]:
#df.columns = df.columns.add_categories(['steroid_status'])
df['steroid_status'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status')
p1.set_xlabel('')
p1.set_ylabel('number of cells')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte_wave2.obs['steroid_status'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status',figsize=(5,5))
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  


p2.text(-0.2,101,'4107',fontsize=9)
p2.text(0.8,101,'21413',fontsize=9)
p2.text(1.8,101,'5033',fontsize=9)
p2.text(2.8,101,'1257',fontsize=9)
p2.text(3.8,101,'12802',fontsize=9)
p2.text(4.8,101,'975',fontsize=9)

# steroid only

In [29]:
# to get sub-samples who got steroid 
steroid = monocyte_wave2[monocyte_wave2.obs['patient_id'].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])]


no_steroid = monocyte_wave2[monocyte_wave2.obs['patient_id'].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190'])]


In [30]:
(list(steroid.obs['case_control'].unique()))

['POSITIVE', 'RECOVERY', 'NEGATIVE']

In [31]:
(list(steroid.obs['patient_id'].unique()))

['C101', 'C138', 'C145', 'C146', 'C147', 'C33']

In [32]:
steroid.obs['discharge_date'].unique()

['21/01/2021', '03/02/2021', '25/01/2021', '26/01/2021', 'NA', '03/06/2021', '11/02/2021']
Categories (7, object): ['03/02/2021', '03/06/2021', '11/02/2021', '21/01/2021', '25/01/2021', '26/01/2021', 'NA']

In [33]:
#correction ##########
steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('str')
steroid.obs.loc[steroid.obs["patient_id"] == "C65", "discharge_date"] = '10/06/2020'
steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('category')

/tmp/ipykernel_5599/590398472.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  steroid.obs['discharge_date'] = steroid.obs['discharge_date'].astype('str')


In [34]:
tmp = steroid.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')



In [35]:
tmp.obs['date_positive_swab'].unique()

array(['02/01/2021', '16/01/2021', '05/01/2021', '09/01/2021', 'NA',
       '12/01/2021'], dtype=object)

In [36]:
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)

In [37]:
tmp.obs['date_first_symptoms'].unique()

array(['06/01/2021', '21/01/2021', '05/01/2021', nan, '14/01/2021'],
      dtype=object)

In [38]:
time_from_first_symptoms = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y')
time_from_positive_swab = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')

In [39]:
time_from_positive_swab.unique()

array([ 345600000000000, 1641600000000000, 1468800000000000,
       1555200000000000, 1209600000000000, 4060800000000000,
        604800000000000,            'NaT', 1382400000000000,
        259200000000000,  172800000000000,  777600000000000,
        864000000000000,  950400000000000, 4665600000000000,
       1036800000000000,  432000000000000], dtype='timedelta64[ns]')

In [40]:
days_from_first_symptoms = time_from_first_symptoms.dt.days
days_from_positive_swab = time_from_positive_swab.dt.days

In [41]:
days_from_positive_swab.unique()

array([ 4., 19., 17., 18., 14., 47.,  7., nan, 16.,  3.,  2.,  9., 10.,
       11., 54., 12.,  5.])

In [42]:
days_from_first_symptoms.fillna(0, inplace = True)
days_from_positive_swab.fillna(0, inplace = True)

/tmp/ipykernel_5599/315576265.py:1: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  days_from_first_symptoms.fillna(0, inplace = True)
/tmp/ipykernel_5599/315576265.py:2: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  days_from_positive_swab.fillna(0, inplace = True)


In [43]:
days_from_positive_swab.unique()

array([ 4., 19., 17., 18., 14., 47.,  7.,  0., 16.,  3.,  2.,  9., 10.,
       11., 54., 12.,  5.])

In [44]:
max(days_from_first_symptoms)


50.0

In [45]:
min(days_from_positive_swab)


0.0

In [46]:
days_from_infection = [max(a,b) for a,b in zip(days_from_first_symptoms, days_from_positive_swab)]


In [47]:
max(days_from_infection)


54.0

In [48]:
steroid.obs['time_from_first_symptoms'] = days_from_first_symptoms
steroid.obs['time_from_positive_swab'] = days_from_positive_swab
steroid.obs['time_from_infection'] = days_from_infection

In [49]:
steroid.obs['final_anno'].unique()

array(['T CD8 Mem', 'T CD4 Helper', 'T CD4 CTL', 'T CD8 CTL',
       'NK CD56 bright', 'NK', 'T CD4 Naive', 'T g/d', 'MAIT', 'T reg',
       'NKT', 'ILC', 'T CD8 Naive', 'T CD8 EMRA'], dtype=object)

In [50]:
# adding dates of steroid inj for each patient
steroid.obs['steroid_date'] = 'nan'
steroid.obs.loc[steroid.obs["patient_id"] == "C146", "steroid_date"] = '22/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C139", "steroid_date"] = '07/02/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C147", "steroid_date"] = '17/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C145", "steroid_date"] = '05/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C138", "steroid_date"] = '28/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C33", "steroid_date"] = '20/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C101", "steroid_date"] = '07/01/2021'
steroid.obs.loc[steroid.obs["patient_id"] == "C141", "steroid_date"] = '02/02/2021'



In [51]:
list(steroid.obs['steroid_date'].unique())

['07/01/2021',
 '28/01/2021',
 '05/01/2021',
 '22/01/2021',
 '17/01/2021',
 '20/01/2021']

In [52]:
tmp = steroid.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
time_from_steroid = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['steroid_date'], format='%d/%m/%Y')


In [53]:
days_from_steroid = time_from_steroid.dt.days

In [54]:
days_from_steroid.max()

49

In [55]:
days_from_steroid.unique()

array([  -1,    7,    5,   18,    2,   34,   -5,   14, -273,   11,  -10,
          9,   -3,    1,   -6, -250,    6, -267, -262,    4,   49,   -4,
         13,    0,   -2,   -8])

In [56]:
steroid.obs['days_from_steroid'] = days_from_steroid

In [57]:
steroid.obs['days_from_steroid'].unique()

array([  -1,    7,    5,   18,    2,   34,   -5,   14, -273,   11,  -10,
          9,   -3,    1,   -6, -250,    6, -267, -262,    4,   49,   -4,
         13,    0,   -2,   -8])

In [58]:
# adding metdata about before and after steroid samples based on date of steroid injection
steroid.obs['steroid_timeline']='nan'

steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'before_steroid_pos'
steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'before_steroid_neg'
steroid.obs.loc[(steroid.obs["days_from_steroid"]<=0)&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'before_steroid_rec'

##########################

steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'after_steroid_pos'
steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'after_steroid_neg'

steroid.obs.loc[(steroid.obs["days_from_steroid"]>0)&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'after_steroid_rec'

steroid.obs['steroid_timeline']=steroid.obs['steroid_timeline'].astype('category')




In [59]:
steroid.obs['steroid_timeline'].unique()

['before_steroid_pos', 'after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg']
Categories (4, object): ['after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg', 'before_steroid_pos']

In [ ]:
#test = steroid[steroid.obs['steroid_timeline']=='after_steroid_neg']
#print(test.obs['case_control'].unique())

In [ ]:
df = pd.crosstab(steroid.obs['steroid_timeline'],steroid.obs['annot4'])
df

In [ ]:
#df.columns = df.columns.add_categories(['steroid_timeline'])
df['steroid_timeline'] = df.index # making index into column

In [ ]:
df

In [ ]:
df=df.reindex(['before_steroid_neg', "before_steroid_pos", "after_steroid_pos", "after_steroid_rec"])
df

In [ ]:
p1=df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
steroid.obs['steroid_timeline'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid',figsize=(3,3))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 


'''
p2.text(-0.2,101,'1257',fontsize=9)
p2.text(0.8,101,'4879',fontsize=9)
p2.text(1.8,101,'7923',fontsize=9)
p2.text(2.8,101,'975',fontsize=9)
'''

#  steroid with positive case only -divided into 4 severity 

In [60]:
steroid.obs['steroid_timeline'].unique()

['before_steroid_pos', 'after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg']
Categories (4, object): ['after_steroid_pos', 'after_steroid_rec', 'before_steroid_neg', 'before_steroid_pos']

In [61]:
steroid_pos_only = steroid[steroid.obs['steroid_timeline'].isin(['before_steroid_pos','after_steroid_pos'])]

In [62]:
steroid_pos_only.obs['WHO_temp_severity'].unique()

['severe', 'mild', 'moderate', 'critical']
Categories (4, object): ['critical', 'mild', 'moderate', 'severe']

In [63]:
# adding severity and steroid meta
steroid_pos_only.obs['steroid_severity']='nan'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'before_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'before_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'before_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'before_steroid_pos_critical'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'after_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'after_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'after_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'after_steroid_pos_critical'


/tmp/ipykernel_5599/1759539440.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  steroid_pos_only.obs['steroid_severity']='nan'


In [64]:
steroid_pos_only.obs['steroid_severity'].unique()

array(['before_steroid_pos_severe', 'after_steroid_pos_mild',
       'after_steroid_pos_moderate', 'before_steroid_pos_moderate',
       'after_steroid_pos_critical', 'before_steroid_pos_mild',
       'after_steroid_pos_severe'], dtype=object)

In [ ]:
#test = steroid_pos_only[steroid_pos_only.obs['steroid_severity']=='after_steroid_pos_severe']
#print(test.obs['case_control'].unique())
#print(test.obs['WHO_temp_severity'].unique())

In [ ]:
df = pd.crosstab(steroid_pos_only.obs['steroid_severity'],steroid_pos_only.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_severity'])
df['steroid_severity'] = df.index 
df

In [ ]:
df=df.reindex(["before_steroid_pos_mild", "after_steroid_pos_mild", "before_steroid_pos_moderate",'after_steroid_pos_moderate','before_steroid_pos_severe','after_steroid_pos_severe',
              'after_steroid_pos_critical'])
df

In [ ]:
p1=df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
steroid_pos_only.obs['steroid_severity'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity',figsize=(5,5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 



p2.text(-0.2,101,'2678',fontsize=9)
p2.text(0.8,101,'1811',fontsize=9)
p2.text(1.8,101,'1911',fontsize=9)
p2.text(2.8,101,'2146',fontsize=9)
p2.text(3.8,101,'290',fontsize=9)
p2.text(4.8,101,'1226',fontsize=9)
p2.text(5.8,101,'2740',fontsize=9)

In [65]:
save_file = '/home/jovyan/scripts/renal_covid_19/scripts_for_publication/trend_of_cell_with_steroid/steroid_pos_only_all_T.h5ad'
steroid_pos_only.write_h5ad(save_file)

In [69]:
list(steroid_pos_only.obs['steroid_status'].unique())

['steroid_positive']